In [1]:
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import nengo
import numpy as np
import tensorflow as tf

import datetime, os
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle
import random

import nengo_dl

# Deep Learning Nengo Network

Tensorflow version 2.2.0 <br>
Nengo dl version 3.0.0

In [2]:
class FeedForwardTrain(pytry.Trial):
    def params(self):
        self.param('run_n', run_n=1)
        self.param('theta', theta=3)
        self.param('q', q=4)
    
    def evaluate(self, param):       
        #LOAD/SET PARAMETERS
        run_n=param.run_n     
        theta=param.theta  
        q=param.q  
        learn_rate=0.000001 #learning rate
        n_epoch=1000 #number of training epochs
        train_rat=0.7 #ratio of training to testing data
        
        print(run_n) #print run number
        np.random.seed() #set seed to random value
        
#################################################################
        #Set and create data directories
        data_dir="D:\\NEN002\\GridSearch_NDL_LMU\\"+str(q)+'_'+str(theta)+"\\" #directory to save data to
        
        if run_n == 0:
            os.mkdir(data_dir)
            os.mkdir("D:\\NEN002\\GridSearch_NDL_LMU_logs\\"+str(q)+'_'+str(theta)+"\\")
            
        logdir = os.path.join("D:\\NEN002\\GridSearch_NDL_LMU_logs\\"+str(q)+'_'+str(theta))+"\\"+str(run_n) #directory to save logs to for tensorboard
        
        #load data
        highengdata = np.load('high_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) #np.load('higheng.npy', allow_pickle=True)
        lowengdata = np.load('low_lmu'+str(q)+'_theta'+str(theta)+'.npy',allow_pickle=True) #np.load('loweng.npy', allow_pickle=True)
##################################################################

        high=list(highengdata[:])
        low=list(lowengdata[:])

        #shuffle clips
        np.random.shuffle(high)
        np.random.shuffle(low)

        #get training clips and convert to list of frames
        high_train = high[:(int(len(low)*train_rat))]
        high_train=list(np.vstack(high_train))
        low_train = low[:(int(len(low)*train_rat))]
        low_train=list(np.vstack(low_train))

        #shuffle frames
        np.random.shuffle(high_train)
        np.random.shuffle(low_train)

        #Make each training set same size (low is shortest)
        high_train = high_train[:(int(len(low_train)))]
        low_train = low_train[:(int(len(low_train)))]

        #Create 10% validation dataset
        high_val = high[(int(len(low)*0.9)):(int(len(low)*1))]
        low_val = low[(int(len(low)*0.9)):(int(len(low)*1))]
        
        #convert to list of frames
        high_val=list(np.vstack(high_val))
        low_val=list(np.vstack(low_val))

        #shuffle frames
        np.random.shuffle(high_val)
        np.random.shuffle(low_val)
        
        #create test sets
        #get 30% of clips for testing
        high_test = high[(int(len(low)*train_rat)):(int(len(low)*0.9))]
        low_test = low[(int(len(low)*train_rat)):(int(len(low)*0.9))]

        #shuffle frames
        np.random.shuffle(high_test)
        np.random.shuffle(low_test)

        #GENERATE INPUT MATRICES
        #Concatenate high and low sets together to create a single array for training, test and validation separately
        all_train = np.vstack(np.concatenate((high_train, low_train)))
        all_test = np.concatenate((high_test, low_test)) 
        all_val = np.concatenate((high_val, low_val))

        #create the target data for training
        target_train = np.zeros((all_train.shape[0],2))
        n_high = len(high_train)
        target_train[:n_high,0] = 1 #target for high = [1,0]
        target_train[n_high:,1] = 1 #target for low = [0,1]
        
        #and for validation
        target_val = np.zeros((all_val.shape[0],2))
        n_high = len(high_val)
        target_val[:n_high,0] = 1
        target_val[n_high:,1] = 1
        
        #SAVE TRAINING AND TESTING DATA
        pickle_filename = (data_dir+"/%s_training_data.pkl") % str(run_n) #(data_dir+"/%s_training_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_train, file)

        pickle_filename = (data_dir+"/%s_testing_data.pkl") % str(run_n) #(data_dir+"/%s_testing_data"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(all_test, file)

        #BUILD MODEL (input layer -> hidden layer -> output layer)
        N = 200
        seed = 1
        N_dims = len(all_train[0]) 

        model = nengo.Network(seed=seed)
        with model:    
            input = nengo.Node(np.zeros(N_dims))
            hidden1 = nengo.Ensemble(n_neurons=N, dimensions=N_dims, radius=np.sqrt(N_dims), 
                                     neuron_type=nengo.RectifiedLinear())
            nengo.Connection(input, hidden1, synapse=None)
            output = nengo.Node(None, size_in=2)
            nengo.Connection(hidden1, output, eval_points=all_train, function=target_train, 
                             scale_eval_points=False, synapse=None)

            p_output = nengo.Probe(output)
        
        #TRAINING DATA AND PARAMETERS
        minibatch_size = 1000       # this can be adjusted to speed up training
                                    # Note: with larger minibatch_size, you may need to reduce this
                                    #(if training is making things worse, then you need to reduce this!)
                                    # number of iterations through the training data to perform

        batches = int(np.ceil(len(all_train)/minibatch_size)) #number of training batches needed (length of data / minibatch size)
        val_batches = int(np.ceil(len(all_val)/minibatch_size)) #number of validation batches needed

        training_data_input = np.array(all_train, copy=True)
        order = np.arange(len(training_data_input))
        np.random.shuffle(order)
        training_data_input_rand = training_data_input[order]
        training_data_input_rand.resize(minibatch_size,batches,N_dims)
        training_data_input.resize(minibatch_size,batches,N_dims)

        training_data_output = np.array(target_train, copy=True)
        training_data_output_rand = training_data_output[order]
        training_data_output.resize(minibatch_size,batches,2)
        training_data_output_rand.resize(minibatch_size,batches,2)
        
        val_input = np.array(all_val, copy=True)
        order = np.arange(len(val_input))
        np.random.shuffle(order)
        val_input_rand = val_input[order]
        val_input_rand.resize(minibatch_size,val_batches,N_dims)
        val_input.resize(minibatch_size,val_batches,N_dims)
        
        val_output = np.array(target_val, copy=True)
        val_output_rand = val_output[order]
        val_output.resize(minibatch_size,val_batches,2)
        val_output_rand.resize(minibatch_size,val_batches,2)
        
        #set learning objective (defines loss)
        def objective(outputs, targets): 
            return tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
                logits=outputs, labels=targets)  
        
        #TRAINING
        with nengo_dl.Simulator(
                model, minibatch_size=minibatch_size) as sim:
            sim.compile(optimizer=tf.optimizers.Adam(learn_rate),
                loss={p_output: objective})
            sim.fit(training_data_input_rand, training_data_output_rand, epochs=n_epoch,
                   callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir)],
                   validation_data=(val_input_rand, val_output_rand))
            sim.save_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n #('./trained.data') 
            
        #Run model with training data, using trained weights
        input.output = nengo.processes.PresentInput(all_train, presentation_time=0.001)
        with nengo_dl.Simulator(model, minibatch_size=1) as sim:
            sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
            sim.run(len(all_train)*0.001)
            
        #Save data
        pickle_filename = (data_dir+"/%s_train_target.pkl") % str(run_n) #(data_dir+"/%s_train_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_train, file)
        
        pickle_filename = (data_dir+"/%s_train_trange.pkl") % str(run_n) #(data_dir+"/%s_train_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_train_out.pkl") % str(run_n) #(data_dir+"/%s_train_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.data[p_output][0], file)    
        
        test_out=[]
        target_test=[]
        for j in range(len(all_test)):
            if j<int(len(high_test)):
                target = np.zeros((all_test[j].shape[0],2))
                target[:,0] = 1
            else:
                target = np.zeros((all_test[j].shape[0],2))
                target[:,1] = 1
            input.output = nengo.processes.PresentInput(all_test[j], presentation_time=0.001)
            with nengo_dl.Simulator(model, minibatch_size=1) as sim:
                sim.load_params(data_dir+"/%s_trained.data" % str(run_n)) #(data_dir+"/%s"+str(test_param)+"_trained.data") % run_n
                sim.run(len(all_test[j])*0.001)
            test_out.append(sim.data[p_output][0])
            target_test.append(target)
            
        #Save data
        pickle_filename = (data_dir+"/%s_test_target.pkl") % str(run_n) #(data_dir+"/%s_test_target"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(target_test, file)
        
        pickle_filename = (data_dir+"/%s_test_trange.pkl") % str(run_n) #(data_dir+"/%s_test_trange"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(sim.trange(), file)
            
        pickle_filename = (data_dir+"/%s_test_out.pkl") % str(run_n) #(data_dir+"/%s_test_out"+str(test_param)+".pkl") % run_n
        with open(pickle_filename, 'wb') as file:
            pickle.dump(test_out, file)


In [ ]:
FeedForwardTrain().run(run_n=7, q=2, theta=3)

running FeedForwardTrain#20200421-150059-632be95d
7
|#####################Building network (57%)                     | ETA: 0:00:00

C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. It is recommended that you install tensorflow-gpu (`pip install tensorflow-gpu`).
  "No GPU support detected. It is recommended that you "


Build finished in 0:00:01                                                      
Optimization finished in 0:00:00                                               
Train on 1000 samples, validate on 1000 samples


Epoch 1/1000
1000/1000 [==============================] - 4s 4ms/sample - loss: 0.7872 - probe_loss: 0.7872 - val_loss: 0.7919 - val_probe_loss: 0.7919
Epoch 2/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.7710 - probe_loss: 0.7710 - val_loss: 0.7757 - val_probe_loss: 0.7757
Epoch 3/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7547 - probe_loss: 0.7547 - val_loss: 0.7594 - val_probe_loss: 0.7594
Epoch 4/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7385 - probe_loss: 0.7385 - val_loss: 0.7432 - val_probe_loss: 0.7432
Epoch 5/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7222 - probe_loss: 0.7222 - val_loss: 0.7270 - val_probe_loss: 0.7270
Epoch 6/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.7060 - probe_loss: 0.7060 - val_loss: 0.7108 - val_probe_loss: 0.7108
Epoch 7/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.6897 -

Epoch 55/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.0902 - probe_loss: -0.0902 - val_loss: -0.0843 - val_probe_loss: -0.0843
Epoch 56/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1064 - probe_loss: -0.1064 - val_loss: -0.1006 - val_probe_loss: -0.1006
Epoch 57/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1227 - probe_loss: -0.1227 - val_loss: -0.1168 - val_probe_loss: -0.1168
Epoch 58/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1389 - probe_loss: -0.1389 - val_loss: -0.1330 - val_probe_loss: -0.1330
Epoch 59/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1552 - probe_loss: -0.1552 - val_loss: -0.1493 - val_probe_loss: -0.1493
Epoch 60/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.1714 - probe_loss: -0.1714 - val_loss: -0.1655 - val_probe_loss: -0.1655
Epoch 61/1000
1000/1000 [==============================] -

Epoch 108/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.9513 - probe_loss: -0.9513 - val_loss: -0.9444 - val_probe_loss: -0.9444
Epoch 109/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.9676 - probe_loss: -0.9676 - val_loss: -0.9606 - val_probe_loss: -0.9606
Epoch 110/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -0.9838 - probe_loss: -0.9838 - val_loss: -0.9768 - val_probe_loss: -0.9768
Epoch 111/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -1.0001 - probe_loss: -1.0001 - val_loss: -0.9930 - val_probe_loss: -0.9930
Epoch 112/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.0163 - probe_loss: -1.0163 - val_loss: -1.0093 - val_probe_loss: -1.0093
Epoch 113/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.0326 - probe_loss: -1.0326 - val_loss: -1.0255 - val_probe_loss: -1.0255
Epoch 114/1000
1000/1000 [==========================

Epoch 160/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.7962 - probe_loss: -1.7962 - val_loss: -1.7881 - val_probe_loss: -1.7881
Epoch 161/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.8125 - probe_loss: -1.8125 - val_loss: -1.8044 - val_probe_loss: -1.8044
Epoch 162/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.8287 - probe_loss: -1.8287 - val_loss: -1.8206 - val_probe_loss: -1.8206
Epoch 163/1000
1000/1000 [==============================] - 2s 2ms/sample - loss: -1.8449 - probe_loss: -1.8449 - val_loss: -1.8368 - val_probe_loss: -1.8368
Epoch 164/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.8612 - probe_loss: -1.8612 - val_loss: -1.8531 - val_probe_loss: -1.8531
Epoch 165/1000
1000/1000 [==============================] - 3s 3ms/sample - loss: -1.8774 - probe_loss: -1.8774 - val_loss: -1.8693 - val_probe_loss: -1.8693
Epoch 166/1000
1000/1000 [==========================

In [ ]:
#for run in range(5):
for theta in (1,3,5,7): #1,3,5,7
    FeedForwardTrain().run(run_n=5, q=2, theta=theta)

To view logs in tensorboard use following command in command prompt: <br>
    tensorboard --logdir={logs_base_dir} --host=localhost <br>
<br>
Note: change {logs_base_dir} to the name of the folder where logs are being saved. <br>
Note: make sure you have navigated to the directory containing this script in the command prompt before running the above command.